In [2]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import cv2


In [3]:
# allows your model to receive new variations of the images at each epoch. 
#But do remember that it only returns the transformed images and 
#does not add it to the set of images that you have
train = ImageDataGenerator(rescale = 1/255)
test = ImageDataGenerator(rescale = 1/255)

In [3]:
#using flow_from_directory to directly import images, and have them classified conveniently :)
training_data = train.flow_from_directory("C:/Users/CZ/Documents/4th semester/Artificial Intelligence/PROJECT/emotions_data/train/",target_size=(200, 200), batch_size = 100, class_mode="categorical")
test_data = test.flow_from_directory("C:/Users/CZ/Documents/4th semester/Artificial Intelligence/PROJECT/emotions_data/test/",target_size=(200, 200), batch_size = 100, class_mode="categorical")

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
#checking if the labels are fine and everything
training_data.class_indices

{'angry': 0,
 'disgusted': 1,
 'fearful': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprised': 6}

In [5]:
#now defining the model. We're basing our model on CNN with the help of tensorflow and maxpooling
#layers are defined in list-form
import tensorflow as tf
num_classes = training_data.num_classes
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3, 3), activation= 'relu', input_shape = (200, 200, 3)),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    #layer change, also increeasing filter size
                                    tf.keras.layers.Conv2D(32, (3, 3), activation= 'relu'),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    #adding layer again
                                    tf.keras.layers.Conv2D(64, (3, 3), activation= 'relu'),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    #now flattening the layer
                                    tf.keras.layers.Flatten(),
                                    #Now applying 2 dense layers
                                    tf.keras.layers.Dense(512, activation= 'relu'),
                                    tf.keras.layers.Dense(num_classes, activation='softmax')])

In [6]:
#now compiling the model
model.compile(loss= 'categorical_crossentropy', optimizer= 'adam', metrics= ['accuracy'])

In [7]:
model_fit = model.fit(training_data, steps_per_epoch=20, epochs=100, validation_data=test_data)
#saving the model so we don't have to train this everytime we boot up
model.save("emotion_model.h5")

Epoch 1/100
20/20 [==============================] - 176s 9s/step - loss: 2.2104 - accuracy: 0.1815 - val_loss: 1.8151 - val_accuracy: 0.2481
Epoch 2/100
20/20 [==============================] - 161s 8s/step - loss: 1.7942 - accuracy: 0.2755 - val_loss: 1.7733 - val_accuracy: 0.2491
Epoch 3/100
20/20 [==============================] - 160s 8s/step - loss: 1.7388 - accuracy: 0.3255 - val_loss: 1.7165 - val_accuracy: 0.3150
Epoch 4/100
20/20 [==============================] - 158s 8s/step - loss: 1.6850 - accuracy: 0.3230 - val_loss: 1.6781 - val_accuracy: 0.3306
Epoch 5/100
20/20 [==============================] - 140s 7s/step - loss: 1.6729 - accuracy: 0.3550 - val_loss: 1.6734 - val_accuracy: 0.3640
Epoch 6/100
20/20 [==============================] - 139s 7s/step - loss: 1.6264 - accuracy: 0.3775 - val_loss: 1.6526 - val_accuracy: 0.3621
Epoch 7/100
20/20 [==============================] - 139s 7s/step - loss: 1.6139 - accuracy: 0.3815 - val_loss: 1.6225 - val_accuracy: 0.3706
Epoch 

In [10]:
#epochs refer to the number of times a model has seen the entire training dataset during the training process
#steps_per_epoch refers to the steps taken before considering one epoch to be completed
#initially we used sigmoid activation, but it was too slow and with terrible accuracy
import tensorflow as tf

#now implementing the camera webcam
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

model = tf.keras.models.load_model("emotion_model.h5")
def emotion_predictor():
     #opening the webcam
     cap = cv2.VideoCapture(0)

     #now to capture stills from the camera and get them checked
     while True:  #so this stays in an infinite loop
          # Read a frame from the webcam
        frame = cap.read()

        # preprocessing the image the model's input shape i.e 200x200
        img = cv2.resize(frame, (200, 200))
        img = np.expand_dims(img, axis=0)
        img = img / 255.0

        # now we predict the captured emotion
        pred = model.predict(img)
        emotion = emotion_labels[np.argmax(pred)]

        # Display the predicted emotion on the frame
        cv2.putText(frame, emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Display the frame
        cv2.imshow('frame', frame)

        # Wait for the user to press the 'q' key to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        



In [ ]:
emotion_predictor()